<a href="https://colab.research.google.com/github/ricardoemf03/applied-physics/blob/main/experimental-physics/muon-mean-lifetime/lab7_muon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar conda
!pip install -q condacolab
import condacolab
condacolab.install()

# Instalar ROOT desde conda-forge
!conda install -c conda-forge root -y

✨🍰✨ Everything looks OK!
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - \ | / - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



##Distancia entre pulsos

In [ ]:
import numpy as np

# Cargar el archivo .npz
data = np.load("/content/drive/MyDrive/ECFM/11/Laboratorio avanzado/lab7  - vida media muon/double_pulses.npz")

# Convertirlo en una lista de arrays (cada array es un pulso con doble caída)
pulses = [data[key] for key in data.files]

print(f"Se cargaron {len(pulses)} pulsos.")


Se cargaron 206 pulsos.


In [ ]:
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

distances = []

for pulse in pulses:
    # Invertir la señal para que los mínimos se conviertan en máximos
    inverted = -pulse

    # Encontrar los picos (mínimos originales)
    peaks, _ = find_peaks(inverted, height=150)  # altura negativa -> altura positiva en señal invertida

    if len(peaks) >= 2:
        # Solo consideramos los dos picos más profundos
        two_main_peaks = sorted(peaks[:2])
        distance = abs(two_main_peaks[1] - two_main_peaks[0])
        distances.append(distance)

# Mostrar algunas distancias
print("Primeras 10 distancias horizontales entre picos:", distances[:10])


Primeras 10 distancias horizontales entre picos: [np.int64(166), np.int64(4), np.int64(239), np.int64(157), np.int64(70), np.int64(448), np.int64(10), np.int64(105), np.int64(95), np.int64(85)]


In [ ]:
import ROOT
from array import array

# Crear histograma ROOT
hist = ROOT.TH1F("distances", "Distancias horizontales entre picos;Distancia (puntos);Frecuencia", 50, 0, max(distances)+10)

# Llenar histograma
for d in distances:
    hist.Fill(d)

# Mostrar el histograma
canvas = ROOT.TCanvas()
hist.Draw()
canvas.Draw()


##Ahora todos lo 6 archivos juntos

In [ ]:
import numpy as np
from scipy.signal import find_peaks
import ROOT
from array import array
import matplotlib.pyplot as plt

# Lista de archivos, direcciones de archivos en drive
archivos = [
    "/content/drive/MyDrive/ECFM/11/Laboratorio avanzado/lab7  - vida media muon/double_pulses1.npz",
    "/content/drive/MyDrive/ECFM/11/Laboratorio avanzado/lab7  - vida media muon/double_pulses2.npz",
    "/content/drive/MyDrive/ECFM/11/Laboratorio avanzado/lab7  - vida media muon/double_pulses3.npz",
    "/content/drive/MyDrive/ECFM/11/Laboratorio avanzado/lab7  - vida media muon/double_pulses4.npz",
    "/content/drive/MyDrive/ECFM/11/Laboratorio avanzado/lab7  - vida media muon/double_pulses5.npz",
    "/content/drive/MyDrive/ECFM/11/Laboratorio avanzado/lab7  - vida media muon/double_pulses6.npz"
]

# Lista para acumular todas las distancias
distances = []

# Iterar sobre cada archivo
for archivo in archivos:
    data = np.load(archivo)
    pulses = [data[key] for key in data.files]

    for pulse in pulses:
        inverted = -pulse
        peaks, _ = find_peaks(inverted, height=150)  # detectar los mínimos originales

        if len(peaks) >= 2:
            # Tomar los dos primeros picos separados (los más cercanos en el eje x)
            two_main_peaks = sorted(peaks[:2])
            distance = abs(two_main_peaks[1] - two_main_peaks[0])
            distances.append(distance*8e-3)

print(f"Se calcularon {len(distances)} distancias.")

# Crear histograma en ROOT
ROOT.gROOT.SetBatch(True)  # No abrir ventanas emergentes en Colab

hist = ROOT.TH1F("distances", "Distancias horizontales entre picos;Distancia (puntos);Frecuencia", 20, 0, max(distances)+1.5)

for d in distances:
    hist.Fill(d)

# Dibujar histograma
canvas = ROOT.TCanvas("canvas", "Histograma", 800, 600)
hist.SetLineColor(ROOT.kBlue + 1)
hist.SetLineWidth(2)
hist.Draw()
canvas.Draw()


Se calcularon 1201 distancias.


Warning in <TROOT::Append>: Replacing existing TH1: distances (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas


##Ajuste exp

In [ ]:
import ROOT

# Definir la función exponencial a ajustar: N0 * exp(-t/tau) + C
fit_function = ROOT.TF1("exp_decay", "[0]*exp(-x/[1]) + [2]", 0, max(distances))

# Estimar valores iniciales aproximados para los parámetros
N0_init = max(hist.GetBinContent(i+1) for i in range(hist.GetNbinsX()))
tau_init = 2.0  # microsegundos
C_init = 0.0

fit_function.SetParameters(N0_init, tau_init, C_init)
fit_function.SetParNames("N_{0}", "#tau", "C")

# Ajustar la función al histograma
hist.Fit("exp_decay", "R")  # R = usar el rango definido en TF1

# Dibujar resultado
canvas = ROOT.TCanvas("canvas", "Ajuste exponencial", 800, 600)
hist.SetTitle("Histograma de tiempos entre pulsos;Tiempo (us);Frecuencia")
hist.SetLineColor(ROOT.kBlue + 1)
hist.SetLineWidth(2)
fit_function.SetLineColor(ROOT.kRed)
fit_function.SetLineWidth(2)
hist.Draw()
fit_function.Draw("same")
canvas.Draw()

# Imprimir parámetros ajustados
N0 = fit_function.GetParameter(0)
tau = fit_function.GetParameter(1)
C = fit_function.GetParameter(2)

print(f"\nResultado del ajuste:")
print(f"N₀ = {N0:.2f}")
print(f"τ (vida media del muón) = {tau:.4f} microsegundos")
print(f"C (offset) = {C:.2f}")



Resultado del ajuste:
N₀ = 259.67
τ (vida media del muón) = 2.1922 microsegundos
C (offset) = 4.42
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      32.9222
NDf                       =           14
Edm                       =  7.67057e-06
NCalls                    =           70
N_{0}                     =      259.669   +/-   12.4181     
#tau                      =      2.19218   +/-   0.1613      
C                         =      4.42451   +/-   3.01882     


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas


##Muestra valores de ajuste en el gráfico

In [ ]:
import ROOT

# Definir función: N0 * exp(-x/tau) + C
fit_function = ROOT.TF1("exp_decay", "[0]*exp(-x/[1]) + [2]", 0, max(distances))
fit_function.SetParameters(N0_init, tau_init, C_init)
fit_function.SetParNames("N_{0}", "#tau", "C")

# Ajuste
hist.Fit("exp_decay", "R")

# Obtener parámetros ajustados y sus errores
N0     = fit_function.GetParameter(0)
tau    = fit_function.GetParameter(1)
C      = fit_function.GetParameter(2)
N0_err = fit_function.GetParError(0)
tau_err = fit_function.GetParError(1)
C_err   = fit_function.GetParError(2)

# Crear canvas
canvas = ROOT.TCanvas("canvas", "Ajuste exponencial", 800, 600)
hist.SetTitle(";Tiempo (μs);Frecuencia (conteos)")
hist.SetLineColor(ROOT.kBlue + 1)
hist.SetLineWidth(2)
fit_function.SetLineColor(ROOT.kRed)
fit_function.SetLineWidth(2)
hist.Draw()
fit_function.Draw("same")

# Mostrar parámetros y errores en el gráfico
ROOT.gStyle.SetOptStat(0)

pave = ROOT.TPaveText(0.6, 0.6, 0.88, 0.73, "NDC")
pave.SetFillColor(0)
pave.SetTextFont(42)
pave.SetTextSize(0.03)
pave.AddText(f"N₀ = {N0:.2f} ± {N0_err:.2f}")
pave.AddText(f"#tau = {tau:.4f} ± {tau_err:.4f} μs")
pave.AddText(f"C = {C:.2f} ± {C_err:.2f}")
pave.Draw()

canvas.Draw()


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      32.9222
NDf                       =           14
Edm                       =  7.67057e-06
NCalls                    =           70
N_{0}                     =      259.669   +/-   12.4181     
#tau                      =      2.19218   +/-   0.1613      
C                         =      4.42451   +/-   3.01882     


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
